# ALI Exams — Minimal Code Pack
Only the essentials: a small **Helpers** cell and then one code cell per exercise.
Each section is titled `Year — Assignment: Name`.

In [4]:

# === Helpers (run once) ===
import numpy as np

def svd(A):
    U,S,Vt = np.linalg.svd(np.asarray(A, dtype=float), full_matrices=True)
    return U,S,Vt

def recon_err(A, U, S, Vt):
    A = np.asarray(A, dtype=float)
    m,n = A.shape
    Sigma = np.zeros((m,n)); np.fill_diagonal(Sigma, S)
    return np.linalg.norm(A - U@Sigma@Vt)

def gs(vectors):
    V = [np.asarray(v, dtype=float).reshape(-1,1) for v in vectors]
    U = []
    for v in V:
        u = v.copy()
        for w in U:
            u -= (w.T@v)/(w.T@w)*w
        if np.linalg.norm(u) > 1e-12: U.append(u)
    return [u.flatten() for u in U]

def gs_on(vectors):
    return [u/np.linalg.norm(u) for u in gs(vectors)]

def fit(X,y):
    X = np.asarray(X, float); y = np.asarray(y, float).reshape(-1,1)
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    yhat = X@beta
    resid = y - yhat
    SSE = float((resid.T@resid).item())
    SST = float(((y - y.mean())**2).sum())
    R2 = 1 - SSE/SST if SST>0 else float("nan")
    return beta.flatten(), yhat.flatten(), SSE, R2


## 2018 Re-Exam — Assignment 5: Gram–Schmidt in ℝ⁴

In [5]:

v1 = [1,0,0,1]; v2 = [1,0,1,1]; v3 = [3,2,1,1]
orth = gs([v1,v2,v3])
on = gs_on([v1,v2,v3])
print("Orthogonal basis:"); [print(u) for u in orth]
print("\nOrthonormal basis:"); [print(u, "||", np.linalg.norm(u)) for u in on]


Orthogonal basis:
[1. 0. 0. 1.]
[0. 0. 1. 0.]
[ 1.  2.  0. -1.]

Orthonormal basis:
[0.70710678 0.         0.         0.70710678] || 0.9999999999999999
[0. 0. 1. 0.] || 1.0
[ 0.40824829  0.81649658  0.         -0.40824829] || 1.0


[None, None, None]

## 2018 Re-Exam — Assignment 6: TV_Viewing (linear vs quadratic)

In [6]:

from scipy.io import loadmat
try:
    mat = loadmat("TV_Viewing.mat")
    arr = np.asarray(mat.get("TV_Viewing", mat.get("data", None)), float)
    age, hours = arr[:,0], arr[:,1]
    Xlin  = np.column_stack([np.ones_like(age), age])
    Xquad = np.column_stack([np.ones_like(age), age, age**2])
    bL, yL, sseL, r2L = fit(Xlin, hours)
    bQ, yQ, sseQ, r2Q = fit(Xquad, hours)
    print("Linear beta:", bL, " SSE:", sseL, " R2:", r2L)
    print("Quadratic beta:", bQ, " SSE:", sseQ, " R2:", r2Q)
    print("Best by SSE:", "Quadratic" if sseQ < sseL else "Linear")
except Exception as e:
    print("Place TV_Viewing.mat next to this notebook. Error:", e)


ModuleNotFoundError: No module named 'scipy'

## 2018 Re-Exam — Assignment 7: SVD of 2×3 matrix

In [ ]:

A = np.array([[3,1,1],
              [1,3,1]], float)
U,S,Vt = svd(A)
print("Singular values:", S)
print("Recon error:", recon_err(A,U,S,Vt))
print("U:\n", U); print("V^T:\n", Vt)


## 2019 Exam — Assignment 2: Span & Dependence

In [ ]:

A = np.array([[ 4,  8, -2],
              [ 6,  2, 10],
              [ 2,  6,  6]], float)
b = np.array([2,18,15], float)
x, *_ = np.linalg.lstsq(A, b, rcond=None)
res = np.linalg.norm(A@x - b)
rank = np.linalg.matrix_rank(A)
print("Least-squares residual:", res, " -> b in Col(A)?", res<1e-10)
print("rank(A) =", rank, " -> columns dependent?" , rank < A.shape[1])


## 2019 Exam — Assignment 3: Determinants (symbolic)

In [ ]:

import sympy as sp
x = sp.symbols('x')
a,b,c = sp.symbols('a b c')
M = sp.Matrix([[a - x, b,     c],
               [1,     -x,    0],
               [0,      1,   -x]])
print("det polynomial:", sp.factor(M.det()))

lam = sp.symbols('lam')
M2 = sp.Matrix([[2 - lam, 4],[3, 2 - lam]])
print("det=0 roots:", sp.solve(sp.Eq(M2.det(),0), lam))


## 2019 Exam — Assignment 4: Orthonormal basis of W ⊆ ℝ⁴

In [ ]:

# Constraint: 2x1 - x3 + x4 = 0  -> choose simple spanning set, then ON it
B = np.column_stack([[1,0,2,0],  # x3=2 -> x1=1
                     [0,1,0,0],  # x2=1
                     [-1,0,0,2]])# x4=2 -> x1=-1
on = gs_on([B[:,0], B[:,1], B[:,2]])
for u in on: print(u, "||", np.linalg.norm(u))


## 2019 Exam — Assignment 6: Smoking & Cancer (choose best, predict at 50)

In [ ]:

import pandas as pd
try:
    df = pd.read_excel("Smoking_and_Cancer.xlsx")
    x = df.iloc[:,0].to_numpy(float); y = df.iloc[:,1].to_numpy(float)
    Xlin  = np.column_stack([np.ones_like(x), x])
    Xquad = np.column_stack([x, x**2])  # no intercept
    bL,_,sseL,r2L = fit(Xlin, y)
    bQ,_,sseQ,r2Q = fit(Xquad,y)
    print("Linear:", bL, " SSE:", sseL, " R2:", r2L)
    print("Quadratic(no c0):", bQ, " SSE:", sseQ, " R2:", r2Q)
    use_quad = sseQ < sseL
    x0 = 50.0
    ypred = (bQ[0]*x0 + bQ[1]*x0**2) if use_quad else (bL[0] + bL[1]*x0)
    print("Best:", "Quadratic" if use_quad else "Linear", " | y(50) =", ypred)
except Exception as e:
    print("Place Smoking_and_Cancer.xlsx next to this notebook. Error:", e)


## 2019 Exam — Assignment 7: SVD of 4×3 matrix

In [ ]:

A = np.array([[2,5,4],
              [6,3,0],
              [6,3,0],
              [2,5,4]], float)
U,S,Vt = svd(A)
print("S:", S); print("Recon error:", recon_err(A,U,S,Vt))


## 2019 Re-Exam — Assignment 3: Spaces & Solution count

In [ ]:

import sympy as sp
A = sp.Matrix([[ 1, -1, 3, 5],
               [-1, -3, 1,-1],
               [ 2,  6,-2, 2]])
print("Nullspace:", [np.array(v, float).flatten() for v in A.nullspace()])
print("Column space:", [np.array(v, float).flatten() for v in A.columnspace()])
print("Row space:", [np.array(v, float).flatten() for v in A.rowspace()])
print("Nullity =", len(A.nullspace()))


## 2019 Re-Exam — Assignment 4: Determinant/cofactor & transforms

In [ ]:

import sympy as sp
a,b = sp.symbols('a b', real=True)
A = sp.Matrix([[-6, a, -1,  3],
               [ 2, 0,  3,  0],
               [ 4, 5,  6,  0],
               [ 8, b,  1, -4]])
print("det(A) =", sp.simplify(A.det()))


## 2019 Re-Exam — Assignment 5: Best approx among candidates

In [ ]:

A = np.array([[2, 2, 1],
              [2,-1, 4],
              [0, 1,-1]], float)
b = np.array([1,1,1], float)
x1 = np.array([1,1,2], float); x2 = np.array([1,1,0], float)
r1 = np.linalg.norm(A@x1 - b); r2 = np.linalg.norm(A@x2 - b)
print("Resid x1:", r1, " | Resid x2:", r2, " -> Best:", "x1" if r1<r2 else "x2")


## 2019 Re-Exam — Assignment 6: Model selection on tiny dataset

In [ ]:

t = np.array([0,2,5,6], float)
y = np.array([1,6,17,19], float)
X1 = np.column_stack([np.ones_like(t), t**2])
X2 = np.column_stack([t, t**2])
X3 = np.column_stack([np.ones_like(t), t, t**2])
for i,(name,X) in enumerate([("y1",X1),("y2",X2),("y3",X3)], start=1):
    b,_,sse,r2 = fit(X,y)
    print(name, "beta:", b, " SSE:", sse, " R2:", r2)


## 2019 Re-Exam — Assignment 7: SVD + U-eigenvector check

In [ ]:

A = np.array([[ 9, 3, 3],
              [-3, 9, 3]], float)
U,S,Vt = svd(A)
AAT = A @ A.T
print("S:", S)
for i in range(U.shape[1]):
    ui = U[:,i]
    lhs = AAT @ ui
    rhs = (S[i]**2)*ui if i < len(S) else np.zeros_like(ui)
    print(f"i={i+1}: ||lhs-rhs|| =", np.linalg.norm(lhs - rhs))


## 2020 Exam (Spring) — Assignment 5: Diagonalize if possible

In [ ]:

import sympy as sp, numpy as np
def diag_try(M):
    M = sp.Matrix(M)
    evects = M.eigenvects()
    cols = []
    for lam, mult, basis in evects:
        cols += basis
    if len(cols) < M.shape[0]:
        print("Not diagonalizable.")
        return
    P = sp.Matrix.hstack(*cols); D = P.inv()*M*P
    print("Diagonalizable. Eigenvalues:", [ev[0] for ev in evects])
A = np.array([[3,2,-2],[0,2,0],[0,1,3]], float)
B = np.array([[-5,2,-1,3],[0,1,0,2],[0,0,1,2],[0,0,0,3]], float)
print("A ->"); diag_try(A)
print("\nB ->"); diag_try(B)


## 2020 Exam (Spring) — Assignment 6: Fit 3 models & velocity at t=4.5

In [ ]:

t = np.arange(13, dtype=float)
y = np.array([0.0,26.4,89.7,186.0,314.1,477.3,666.0,883.5,1141.2,1413.3,1715.1,1715.1,2427.6], float)

Xc = np.column_stack([np.ones_like(t), t, t**2, t**3])
Xc0= np.column_stack([t, t**2, t**3])
Xq = np.column_stack([np.ones_like(t), t, t**2])

results = []
for name, X in [("cubic_with_c0",Xc), ("cubic_no_c0",Xc0), ("quadratic",Xq)]:
    b,_,sse,r2 = fit(X,y)
    results.append((name,b,sse,r2)); print(name, "SSE:", sse, "R2:", r2)
best = min(results, key=lambda z: z[2])
print("\nBest:", best[0])
t0 = 4.5; b = best[1]
if best[0]=="quadratic":
    d0,d1,d2 = b; vel = d1 + 2*d2*t0
elif best[0]=="cubic_no_c0":
    g1,g2,g3 = b; vel = g1 + 2*g2*t0 + 3*g3*t0**2
else:
    b0,b1,b2,b3 = b; vel = b1 + 2*b2*t0 + 3*b3*t0**2
print("Velocity at t=4.5:", vel)


## 2020 Exam (Spring) — Assignment 7: SVD + subspaces

In [ ]:

A = np.array([[2,0,0],[0,2,1],[0,1,2],[0,0,0]], float)
U,S,Vt = svd(A)
m,n = A.shape
Sigma = np.zeros((m,n)); np.fill_diagonal(Sigma, S)
print("S:", S, " | recon err:", np.linalg.norm(A - U@Sigma@Vt))
# Bases from SVD
r = np.sum(S>1e-10)
print("rank:", int(r))
print("Col(A) = span of first r columns of U:\n", U[:, :r])
print("Nul(A) = span of last n-r columns of V:\n", Vt[r:, :].T)
print("Nul(A^T) = span of last m-r columns of U:\n", U[:, r:])
